In [1]:
import boto3
import json
from PIL import Image
import io

# Set up the SageMaker client
sagemaker_client = boto3.client('sagemaker-runtime')

# Set up the endpoint name and the input data
endpoint_name = "huggingface-pytorch-inference-2023-05-02-00-15-37-215"


In [2]:
# Load the local image
image_path = 'tiger.jpg'
with open(image_path, 'rb') as f:
    image_bytes = f.read()

# Convert the image to bytes
with io.BytesIO(image_bytes) as buffer:
    image = Image.open(buffer)
    image.save(buffer, format='JPEG')
    image_bytes = buffer.getvalue()

# Send the request to the endpoint
response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='image/jpeg',
    Body=image_bytes
)

# Get the response
result = response['Body'].read().decode('utf-8')

In [24]:
import base64
import json
import boto3
import os
import requests

with open("tiger.jpg", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

# Create a test event
event = {"body": encoded_image}

# Invoke the lambda function
client = boto3.client("lambda")
response = client.invoke(
    FunctionName="classify",
    Payload=json.dumps(event)
)


# Print the response from the lambda function
#print(response["Payload"].read().decode("utf-8"))

response_dict = json.loads(response["Payload"].read().decode("utf-8"))
print(json.loads(response_dict.get('body')))

{'statusCode': 200, 'body': '[{"score":0.75298672914505,"label":"tiger, Panthera tigris"},{"score":0.13333521783351898,"label":"tiger cat"},{"score":0.0005143850576132536,"label":"lynx, catamount"},{"score":0.000501271104440093,"label":"jaguar, panther, Panthera onca, Felis onca"},{"score":0.00030192412668839097,"label":"theater curtain, theatre curtain"}]'}


In [36]:
json.loads(response_dict.get('body'))

[{'score': 0.75298672914505, 'label': 'tiger, Panthera tigris'},
 {'score': 0.13333521783351898, 'label': 'tiger cat'},
 {'score': 0.0005143850576132536, 'label': 'lynx, catamount'},
 {'score': 0.000501271104440093,
  'label': 'jaguar, panther, Panthera onca, Felis onca'},
 {'score': 0.00030192412668839097,
  'label': 'theater curtain, theatre curtain'}]

In [31]:
predictions = response_dict.get("body")

# Format and print the prediction results
if len(predictions) > 0:
    print("Predictions:")
    for pred in predictions:
        class_name = pred.get("label", {}).get("display_name", "Unknown")
        score = pred.get("label", {}).get("score", 0.0)
        print(f"{class_name}: {score:.2f}")
else:
    print("No predictions found.")

Predictions:


AttributeError: 'str' object has no attribute 'get'